In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=7282e4d3c5d92c1f35dcbdd07367a0ba93f6d2c4fd9a8fc1353caac28a7c3971
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [ ]:
import pandas as pd

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.ml.feature import Tokenizer, HashingTF, IDF
from pyspark.ml.classification import LogisticRegression

# Build a spark context
hc = (SparkSession.builder
                  .appName('Toxic Comment Classification')
                  .enableHiveSupport()
                  .config("spark.executor.memory", "4G")
                  .config("spark.driver.memory","18G")
                  .config("spark.executor.cores","7")
                  .config("spark.python.worker.memory","4G")
                  .config("spark.driver.maxResultSize","0")
                  .config("spark.sql.crossJoin.enabled", "true")
                  .config("spark.serializer","org.apache.spark.serializer.KryoSerializer")
                  .config("spark.default.parallelism","2")
                  .getOrCreate())

hc.sparkContext.setLogLevel('INFO')
hc.version

'3.4.0'

In [ ]:
import pandas as pd

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.ml.feature import Tokenizer, HashingTF, IDF
from pyspark.ml.classification import LogisticRegression

hc = (SparkSession.builder
                  .appName('Toxic Comment Classification')
                  .enableHiveSupport()
                  .config("spark.executor.memory", "4G")
                  .config("spark.driver.memory","18G")
                  .config("spark.executor.cores","7")
                  .config("spark.python.worker.memory","4G")
                  .config("spark.driver.maxResultSize","0")
                  .config("spark.sql.crossJoin.enabled", "true")
                  .config("spark.serializer","org.apache.spark.serializer.KryoSerializer")
                  .config("spark.default.parallelism","2")
                  .getOrCreate())

def to_spark_df(fin):
    try:
        df = pd.read_csv(fin, encoding='latin1', error_bad_lines=False)
    except pd.errors.ParserError as e:
        print(f"Skipping problematic rows in file {fin}...")
        df = pd.read_csv(fin, encoding='latin1', error_bad_lines=False, quoting=csv.QUOTE_NONE)

    df = df.dropna()
    df = df.drop_duplicates()
    df = df.astype(str)
    sdf = hc.createDataFrame(df)
    sdf = sdf.select([F.col(col).alias(col.strip().replace(" ", "_")) for col in sdf.columns])
    return sdf

# Load the train-test sets
train = to_spark_df("/content/train.csv")
test = to_spark_df("/content/test.csv")



<ipython-input-2-9305b5f92fe9>:24: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(fin, encoding='latin1', error_bad_lines=False)
<ipython-input-2-9305b5f92fe9>:24: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(fin, encoding='latin1', error_bad_lines=False)


In [ ]:
def to_spark_df(fin):
    try:
        df = pd.read_csv(fin, encoding='latin1', error_bad_lines=False)
    except pd.errors.ParserError as e:
        print(f"Skipping problematic rows in file {fin}...")
        df = pd.read_csv(fin, encoding='latin1', error_bad_lines=False, quoting=csv.QUOTE_NONE)

    df = df.dropna()
    df = df.drop_duplicates()
    df = df.astype(str)
    sdf = hc.createDataFrame(df)
    sdf = sdf.select([F.col(col).alias(col.strip().replace(" ", "_")) for col in sdf.columns])
    return sdf

# Load the train-test sets
train = to_spark_df("/content/train.csv")
test = to_spark_df("/content/test.csv")

<ipython-input-3-e84907e942ba>:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(fin, encoding='latin1', error_bad_lines=False)
<ipython-input-3-e84907e942ba>:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(fin, encoding='latin1', error_bad_lines=False)


In [ ]:
train.show(5)
test.show(5)

+----------------+--------------------+-----+------------+-------+------+------+-------------+
|              id|        comment_text|toxic|severe_toxic|obscene|threat|insult|identity_hate|
+----------------+--------------------+-----+------------+-------+------+------+-------------+
|0000997932d777bf|Explanation\nWhy ...|    0|           0|      0|     0|     0|            0|
|000103f0d9cfb60f|D'aww! He matches...|    0|           0|      0|     0|     0|            0|
|000113f07ec002fd|Hey man, I'm real...|    0|           0|      0|     0|     0|            0|
|0001b41b1c6bb37e|"\nMore\nI can't ...|    0|           0|      0|     0|     0|            0|
|0001d958c54c6e35|You, sir, are my ...|    0|           0|      0|     0|     0|            0|
+----------------+--------------------+-----+------------+-------+------+------+-------------+
only showing top 5 rows

+----------------+--------------------+
|              id|        comment_text|
+----------------+--------------------+


In [ ]:
out_cols = [i for i in train.columns if i not in ["id", "comment_text"]]
train.show(5)
# View some toxic comments
train.filter(F.col('toxic') == 1).show(5)

+----------------+--------------------+-----+------------+-------+------+------+-------------+
|              id|        comment_text|toxic|severe_toxic|obscene|threat|insult|identity_hate|
+----------------+--------------------+-----+------------+-------+------+------+-------------+
|0000997932d777bf|Explanation\nWhy ...|    0|           0|      0|     0|     0|            0|
|000103f0d9cfb60f|D'aww! He matches...|    0|           0|      0|     0|     0|            0|
|000113f07ec002fd|Hey man, I'm real...|    0|           0|      0|     0|     0|            0|
|0001b41b1c6bb37e|"\nMore\nI can't ...|    0|           0|      0|     0|     0|            0|
|0001d958c54c6e35|You, sir, are my ...|    0|           0|      0|     0|     0|            0|
+----------------+--------------------+-----+------------+-------+------+------+-------------+
only showing top 5 rows

+----------------+--------------------+-----+------------+-------+------+------+-------------+
|              id|       

In [ ]:
# Basic sentence tokenizer
tokenizer = Tokenizer(inputCol="comment_text", outputCol="words")
wordsData = tokenizer.transform(train)
# Count the words in a document
hashingTF = HashingTF(inputCol="words", outputCol="rawFeatures")
tf = hashingTF.transform(wordsData)
tf.select('rawFeatures').take(2)


[Row(rawFeatures=SparseVector(262144, {6240: 1.0, 7221: 1.0, 9420: 1.0, 10214: 1.0, 11680: 1.0, 15494: 1.0, 19036: 1.0, 19208: 1.0, 23032: 1.0, 25000: 1.0, 26144: 1.0, 66299: 1.0, 67416: 1.0, 72125: 1.0, 74944: 1.0, 77971: 1.0, 79300: 1.0, 79968: 1.0, 89833: 1.0, 94488: 1.0, 95889: 3.0, 97171: 1.0, 101169: 1.0, 103863: 1.0, 110427: 1.0, 110510: 1.0, 116767: 1.0, 140784: 1.0, 141086: 1.0, 145284: 1.0, 151536: 1.0, 151751: 1.0, 166368: 1.0, 187114: 1.0, 219915: 1.0, 223402: 1.0, 229137: 1.0, 231630: 1.0, 233967: 1.0, 240944: 1.0, 253170: 1.0})),
 Row(rawFeatures=SparseVector(262144, {2195: 1.0, 4714: 1.0, 13283: 1.0, 48234: 1.0, 85939: 1.0, 108541: 1.0, 119702: 1.0, 121320: 1.0, 137179: 1.0, 141086: 1.0, 159767: 1.0, 165258: 1.0, 169800: 1.0, 212492: 1.0, 218233: 1.0, 224255: 1.0, 224850: 1.0, 249180: 1.0}))]

In [ ]:
# Build the idf model and transform the original token frequencies into their tf-idf counterparts
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(tf) 
tfidf = idfModel.transform(tf)
tfidf.select("features").first()

Row(features=SparseVector(262144, {6240: 8.7614, 7221: 2.2031, 9420: 3.1544, 10214: 6.4668, 11680: 5.0285, 15494: 3.4225, 19036: 0.7388, 19208: 2.2452, 23032: 5.0123, 25000: 5.6905, 26144: 3.5882, 66299: 7.7906, 67416: 1.1955, 72125: 2.2737, 74944: 2.5143, 77971: 7.6235, 79300: 6.6769, 79968: 9.9008, 89833: 3.0525, 94488: 8.4249, 95889: 1.2134, 97171: 2.0171, 101169: 1.735, 103863: 6.8444, 110427: 2.1179, 110510: 5.6685, 116767: 6.0244, 140784: 3.0501, 141086: 2.4784, 145284: 8.0884, 151536: 2.2433, 151751: 9.0358, 166368: 2.0436, 187114: 1.7668, 219915: 0.697, 223402: 3.3526, 229137: 4.5711, 231630: 9.4953, 233967: 3.103, 240944: 1.7546, 253170: 2.7016}))

In [ ]:
REG = 0.1
lr = LogisticRegression(featuresCol="features", labelCol='toxic', regParam=REG)
tfidf.show(5)

+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+--------------------+
|              id|        comment_text|toxic|severe_toxic|obscene|threat|insult|identity_hate|               words|         rawFeatures|            features|
+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+--------------------+
|0000997932d777bf|Explanation\nWhy ...|    0|           0|      0|     0|     0|            0|[explanation, why...|(262144,[6240,722...|(262144,[6240,722...|
|000103f0d9cfb60f|D'aww! He matches...|    0|           0|      0|     0|     0|            0|[d'aww!, he, matc...|(262144,[2195,471...|(262144,[2195,471...|
|000113f07ec002fd|Hey man, I'm real...|    0|           0|      0|     0|     0|            0|[hey, man,, i'm, ...|(262144,[18700,27...|(262144,[18700,27...|
|0001b41b1c6bb37e|"\nMore\nI can't ...|    0|       

In [ ]:
from pyspark.sql.functions import col
tfidf = tfidf.withColumn("toxic", col("toxic").cast("double"))
lrModel = lr.fit(tfidf.limit(4000))
res_train = lrModel.transform(tfidf)
res_train.select("id", "toxic", "probability", "prediction").show(10)
res_train.show(5)

+----------------+-----+--------------------+----------+
|              id|toxic|         probability|prediction|
+----------------+-----+--------------------+----------+
|0000997932d777bf|  0.0|[0.98708414415883...|       0.0|
|000103f0d9cfb60f|  0.0|[0.98578386377281...|       0.0|
|000113f07ec002fd|  0.0|[0.95195125534514...|       0.0|
|0001b41b1c6bb37e|  0.0|[0.99406923917414...|       0.0|
|0001d958c54c6e35|  0.0|[0.96920502028432...|       0.0|
|00025465d4725e87|  0.0|[0.96326972334709...|       0.0|
|0002bcb3da6cb337|  1.0|[0.28384026420489...|       1.0|
|00031b1e95af7921|  0.0|[0.96521315191035...|       0.0|
|00037261f536c51d|  0.0|[0.98623994566883...|       0.0|
|00040093b2687caa|  0.0|[0.97061660492107...|       0.0|
+----------------+-----+--------------------+----------+
only showing top 10 rows

+----------------+--------------------+-----+------------+-------+------+------+-------------+--------------------+--------------------+--------------------+-------------------

In [ ]:
extract_prob = F.udf(lambda x: float(x[1]), T.FloatType())
(res_train.withColumn("proba", extract_prob("probability"))
 .select("proba", "prediction").show())

+------------+----------+
|       proba|prediction|
+------------+----------+
| 0.012915856|       0.0|
| 0.014216136|       0.0|
| 0.048048746|       0.0|
| 0.005930761|       0.0|
|  0.03079498|       0.0|
|  0.03673028|       0.0|
|  0.71615976|       1.0|
| 0.034786846|       0.0|
|0.0137600545|       0.0|
| 0.029383395|       0.0|
| 8.825276E-6|       0.0|
| 0.028142199|       0.0|
|   0.9568293|       1.0|
|0.0036047543|       0.0|
| 0.017964894|       0.0|
| 0.005057816|       0.0|
|   0.8439702|       1.0|
| 0.023082986|       0.0|
| 0.019967748|       0.0|
| 0.012109306|       0.0|
+------------+----------+
only showing top 20 rows



In [ ]:
test_tokens = tokenizer.transform(test)
test_tf = hashingTF.transform(test_tokens)
test_tfidf = idfModel.transform(test_tf)

test_res = test.select('id')
test_res.head()

Row(id='00001cee341fdb12')

In [ ]:
from pyspark.sql.functions import col

tfidf = tfidf.withColumn("severe_toxic", col("severe_toxic").cast("double"))
tfidf = tfidf.withColumn("obscene", col("obscene").cast("double"))
tfidf = tfidf.withColumn("threat", col("threat").cast("double"))
tfidf = tfidf.withColumn("insult", col("insult").cast("double"))
tfidf = tfidf.withColumn("identity_hate", col("identity_hate").cast("double"))

In [ ]:

test_probs = []
for col in out_cols:
    print(col)
    lr = LogisticRegression(featuresCol="features", labelCol=col, regParam=REG)
    print("...fitting")
    lrModel = lr.fit(tfidf)
    print("...predicting")
    res = lrModel.transform(test_tfidf)
    print("...appending result")
    test_res = test_res.join(res.select('id', 'probability'), on="id")
    print("...extracting probability")
    test_res = test_res.withColumn(col, extract_prob('probability')).drop("probability")
    test_res.show(5)

toxic
...fitting
...predicting
...appending result
...extracting probability
+----------------+-----------+
|              id|      toxic|
+----------------+-----------+
|000968ce11f5ee34|0.047582276|
|00491682330fdd1d|3.734712E-9|
|008eb47c4684d190|  0.9069167|
|00d251f47486b6d2| 0.06388117|
|0114ae82c53101a9|  0.5112876|
+----------------+-----------+
only showing top 5 rows

severe_toxic
...fitting
...predicting
...appending result
...extracting probability
+----------------+-----------+------------+
|              id|      toxic|severe_toxic|
+----------------+-----------+------------+
|000968ce11f5ee34|0.047582276|0.0077139614|
|00491682330fdd1d|3.734712E-9|2.5387094E-6|
|008eb47c4684d190|  0.9069167|0.0013850711|
|00d251f47486b6d2| 0.06388117| 0.007744039|
|0114ae82c53101a9|  0.5112876|  0.07452955|
+----------------+-----------+------------+
only showing top 5 rows

obscene
...fitting
...predicting
...appending result
...extracting probability
+----------------+-----------+-----

In [ ]:
test_res.coalesce(1).write.csv('/content/sample_data/spark_lr.csv', mode='overwrite', header=True)


In [ ]:
!cat /content/sample_data/spark_lr.csv/part*.csv > spark_lr.csv